In [ ]:
pip install qiskit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 13.8 MB/s eta 0:00:00


In [ ]:
import sklearn
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 61.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
import pandas as pd
from torch.nn.functional import relu

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pennylane as qml
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
from pennylane.operation import Tensor

import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/content/Wine.csv")

In [ ]:
df.head()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1


In [ ]:
#Using only a sample from dataset
df1 = df.iloc[0:178,0:]

In [ ]:
df1.shape

(178, 14)

In [ ]:
labels = df['Customer_Segment']
df1.drop('Customer_Segment',axis=1,inplace=True)

In [ ]:
X = df1.to_numpy()

In [ ]:

n_qubits = 13

In [ ]:

dev_kernel = qml.device("default.qubit", wires=n_qubits)

projector = np.zeros((2**n_qubits, 2**n_qubits))
projector[0, 0] = 1

@qml.qnode(dev_kernel)
def kernel(x1, x2):
    AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size=0.05, random_state=0)

In [ ]:
kernel(X_train[0],X_train)

tensor([1.00000000e+00, 2.98365778e-05, 1.50138490e-06, 8.49264966e-06,
        1.99737621e-03, 2.53096461e-01, 2.15951397e-01, 3.68191252e-04,
        1.16975187e-03, 1.48884815e-03, 5.62710142e-05, 4.94668383e-12,
        4.70432839e-04, 4.58592552e-04, 7.69189083e-04, 2.01776314e-02,
        1.10142166e-02, 6.90998176e-05, 1.52030858e-03, 2.31808076e-06,
        5.17394279e-07, 1.79952986e-03, 1.27026478e-03, 2.11366364e-03,
        9.34488809e-08, 2.77224793e-06, 7.94918169e-05, 1.55754488e-03,
        1.47733788e-02, 2.16569312e-04, 5.97401518e-03, 6.93742280e-03,
        5.58437382e-05, 1.69937864e-01, 6.44486837e-01, 1.07690841e-02,
        1.14698258e-06, 1.49440010e-01, 1.87038507e-03, 5.36312659e-02,
        5.46366517e-04, 2.33233741e-04, 5.65220415e-04, 3.48342481e-04,
        1.77202992e-03, 3.16694698e-03, 1.23467498e-01, 2.18236027e-12,
        7.84158438e-04, 4.28188660e-02, 1.08562826e-06, 1.65358565e-02,
        1.01739113e-01, 3.98473247e-05, 1.47005322e-04, 3.499355

In [ ]:
def kernel_matrix(A, B):
    return np.array([[kernel(a, b) for b in B] for a in A])

In [ ]:
svm = SVC(kernel=kernel_matrix).fit(X_train, y_train)

In [ ]:
predictions = svm.predict(Xtest)
accuracy_score(predictions, ytest)

0.9375

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
report = classification_report(ytest, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        15
           1       0.00      0.00      0.00         1

    accuracy                           0.94        16
   macro avg       0.47      0.50      0.48        16
weighted avg       0.88      0.94      0.91        16



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
